# Naver blog scrap

## 라이브러리

In [1082]:
import pandas as pd
import time, os, random
from datetime import datetime
from numpy import nan

In [1083]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import chromedriver_autoinstaller as ca

## 크롤링 함수

In [1084]:
def crawling(comment_areas, list):
    # 리스트 지정
    date_list = list[0]
    rare_number_list = list[1]
    check_reply_list = list[2]
    nickname_list = list[3]
    writer_url_list = list[4]
    like_num_list = list[5]
    link_num_list = list[6]
    reply_num_list = list[7]
    img_list = list[8]
    comment_list = list[9]

    for one in comment_areas:
        # 작성일 수집
        date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
        date_list.append(date)

        # 고유번호
        temp = one.attrs['class'][1].split('_')[-1]
        rare_number_list.append(temp)

        # 답글 구분
        try:
            one.find('span', 'u_cbox_secret_contents').text
            temp = '비밀댓글'
        except:
            try:
                temp = one.find('div', 'u_cbox_reply_area').find('li')
                temp = nan
            except:
                temp = '답글'
        check_reply_list.append(temp)

        # 닉네임
        try:
            temp = one.find('span', 'u_cbox_nick').text
        except:
            temp = '비밀댓글'
        nickname_list.append(temp)

        # 작성자 url
        try:
            temp = one.find('a', 'u_cbox_name').attrs['href']
        except:
            temp = '비밀댓글'
        writer_url_list.append(temp)

        # 공감수
        try:
            temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
        except:
            temp = 0
        like_num_list.append(temp)

        # 링크 수
        try:
            temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
            temp = len(temp)
        except:
            temp = 0
        link_num_list.append(temp)

        # 답글 수
        try:
            temp = one.find('div', 'u_cbox_reply_area').find_all('li')
            temp = len(temp)
        except:
            temp = 0
        reply_num_list.append(temp)

        # 첨부이미지url
        try:
            temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
        except:
            temp = nan
        img_list.append(temp)

        # 댓글 내용 수집
        try:
            temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
        except:
            temp = '비밀댓글'
        comment_list.append(temp)

    # 수집한 데이터 확인
    print('작성일 리스트 길이 : ', len(date_list))
    print('고유번호 리스트 길이 : ', len(rare_number_list))
    print('답글구분 리스트 길이 : ', len(check_reply_list))
    print('닉네임 리스트 길이 : ', len(nickname_list))
    print('작성자URL 리스트 길이 : ', len(writer_url_list))
    print('공감수 리스트 길이 : ', len(like_num_list))
    print('링크수 리스트 길이 : ', len(link_num_list))
    print('답글수 리스트 길이 : ', len(reply_num_list))
    print('첨부이미지URL 리스트 길이 : ', len(img_list))
    print('내용 리스트 길이 : ', len(comment_list))

    # 리스트에 담기
    crawled_list = [date_list, rare_number_list, check_reply_list, nickname_list, 
            writer_url_list, like_num_list, link_num_list, reply_num_list,
            img_list, comment_list]

    return crawled_list

## 현재 파일 위치 확인

In [1085]:
# 현재 파일 위치 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/incheon'

## 수집한 파일 저장 폴더 생성

In [1086]:
# 수집한 파일을 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/naver_blog'
os.makedirs(crawled_folder_path, exist_ok=True)

## 크롬 드라이버 버전 확인 및 설치

In [1087]:
# 크롬 드라이버 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'100'

In [1088]:
# 크롬 드라이버 설치 확인
try:
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')
except:
    ca.install(True)
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3261653714.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')


## 페이지 접속

In [1089]:
# 페이지 접속
url = 'https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222657505998&categoryNo=9&parentCategoryNo=9&from=thumbnailList'
driver.get(url)
driver.implicitly_wait(3)
driver.maximize_window()

In [1090]:
# page 정보
page_info = url.split('=')[2].split('&')[0]
page_info

'222657505998'

## 페이지 스크래핑

### 댓글 페이지 개수

In [1091]:
# 댓글 버튼 클릭
comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')
comment_button.click()
time.sleep( random.uniform(0.5, 1) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/4256789063.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')


In [1092]:
soup = bs(driver.page_source, 'lxml')
num_page_buttons = int(soup.find('span', '_lastPageNo').text)
num_page_buttons

15

In [1093]:
soup.title.text

"[이벤트] '인천광역시의회를 찾아라!' : 네이버 블로그"

In [1094]:
if num_page_buttons <= 10:
    prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')
    try:
        prev_button.click()
        time.sleep( random.uniform(0.5, 1) )
    except:
        pass
elif num_page_buttons > 10:
    prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')
    for i in range(num_page_buttons // 10 + 1):
        try:
            prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')
            prev_button.click()
            time.sleep( random.uniform(0.5, 1) )
        except:
            pass

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/1503457795.py:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/1503457795.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')


In [1095]:
list = [[], [], [], [], [], [], [], [], [], []]
len(list), list

(10, [[], [], [], [], [], [], [], [], [], []])

In [1096]:
# 페이지 이동하며 정보 크롤링
for i in range(num_page_buttons - 1):
    # 페이지 수프에 담기
    comment_page_soup = bs(driver.page_source, 'lxml')
    comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
    num_current_page = int( comment_page_soup.find('strong', '_currentPageNo').text )
    print( f"{num_current_page} 페이지 댓글 수 :", len(comment_areas) )
    # 정보 크롤링
    list = crawling(comment_areas, list)

    try:
        # 페이지 이동
        button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}_ct"]/div[1]/div/div[2]/a[2]')
        button.click()
        driver.implicitly_wait(1)
        time.sleep( random.uniform(0.5, 1) )

        # 페이지 수프에 담기
        comment_page_soup = bs(driver.page_source, 'lxml')
        comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
        num_current_page = int( comment_page_soup.find('strong', '_currentPageNo').text )
        print( f"{num_current_page} 페이지 댓글 수 :", len(comment_areas) )
        # 정보 크롤링
        list = crawling(comment_areas, list)
    except:
        pass

1 페이지 댓글 수 : 50
작성일 리스트 길이 :  50
고유번호 리스트 길이 :  50
답글구분 리스트 길이 :  50
닉네임 리스트 길이 :  50
작성자URL 리스트 길이 :  50
공감수 리스트 길이 :  50
링크수 리스트 길이 :  50
답글수 리스트 길이 :  50
첨부이미지URL 리스트 길이 :  50
내용 리스트 길이 :  50


C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/628521845.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}_ct"]/div[1]/div/div[2]/a[2]')


2 페이지 댓글 수 : 50
작성일 리스트 길이 :  100
고유번호 리스트 길이 :  100
답글구분 리스트 길이 :  100
닉네임 리스트 길이 :  100
작성자URL 리스트 길이 :  100
공감수 리스트 길이 :  100
링크수 리스트 길이 :  100
답글수 리스트 길이 :  100
첨부이미지URL 리스트 길이 :  100
내용 리스트 길이 :  100
2 페이지 댓글 수 : 50
작성일 리스트 길이 :  150
고유번호 리스트 길이 :  150
답글구분 리스트 길이 :  150
닉네임 리스트 길이 :  150
작성자URL 리스트 길이 :  150
공감수 리스트 길이 :  150
링크수 리스트 길이 :  150
답글수 리스트 길이 :  150
첨부이미지URL 리스트 길이 :  150
내용 리스트 길이 :  150
3 페이지 댓글 수 : 50
작성일 리스트 길이 :  200
고유번호 리스트 길이 :  200
답글구분 리스트 길이 :  200
닉네임 리스트 길이 :  200
작성자URL 리스트 길이 :  200
공감수 리스트 길이 :  200
링크수 리스트 길이 :  200
답글수 리스트 길이 :  200
첨부이미지URL 리스트 길이 :  200
내용 리스트 길이 :  200
3 페이지 댓글 수 : 50
작성일 리스트 길이 :  250
고유번호 리스트 길이 :  250
답글구분 리스트 길이 :  250
닉네임 리스트 길이 :  250
작성자URL 리스트 길이 :  250
공감수 리스트 길이 :  250
링크수 리스트 길이 :  250
답글수 리스트 길이 :  250
첨부이미지URL 리스트 길이 :  250
내용 리스트 길이 :  250
4 페이지 댓글 수 : 50
작성일 리스트 길이 :  300
고유번호 리스트 길이 :  300
답글구분 리스트 길이 :  300
닉네임 리스트 길이 :  300
작성자URL 리스트 길이 :  300
공감수 리스트 길이 :  300
링크수 리스트 길이 :  300
답글수 리스트 길이 :  300
첨부이미지URL 리스트 길이

## 수집한 데이터 확인

In [1097]:
print('작성일 리스트 길이 : ', len(list[0]))
print('고유번호 리스트 길이 : ', len(list[1]))
print('답글구분 리스트 길이 : ', len(list[2]))
print('닉네임 리스트 길이 : ', len(list[3]))
print('작성자URL 리스트 길이 : ', len(list[4]))
print('공감수 리스트 길이 : ', len(list[5]))
print('링크수 리스트 길이 : ', len(list[6]))
print('답글수 리스트 길이 : ', len(list[7]))
print('첨부이미지URL 리스트 길이 : ', len(list[8]))
print('내용 리스트 길이 : ', len(list[9]))

작성일 리스트 길이 :  1382
고유번호 리스트 길이 :  1382
답글구분 리스트 길이 :  1382
닉네임 리스트 길이 :  1382
작성자URL 리스트 길이 :  1382
공감수 리스트 길이 :  1382
링크수 리스트 길이 :  1382
답글수 리스트 길이 :  1382
첨부이미지URL 리스트 길이 :  1382
내용 리스트 길이 :  1382


## df 만들기

In [1098]:
dict = {'작성일':list[0], '고유번호':list[1], 
        '답글구분':list[2], '닉네임':list[3], 
        '작성자URL':list[4], '공감수':list[5], 
        '링크수':list[6], '답글수':list[7], 
        '첨부이미지URL':list[8], '내용':list[9]}
df = pd.DataFrame(dict)

In [1099]:
df.head()

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
0,2022-02-25 12:58:42,750627914109682094,NaN,문득,https://blog.naver.com/ekdlskdl,0,0,0,NaN,*정답: 3번 @해피쏭
1,2022-02-25 12:59:29,750627965884170523,NaN,푸른하늘현,https://blog.naver.com/blueday118,0,1,0,NaN,정답 : 3번 https://blog.naver.com/blueday118/2226...
2,2022-02-25 13:05:52,750628376070324774,NaN,란,https://blog.naver.com/cdism,0,1,0,NaN,*정답: 3번 https://blog.naver.com/cdism @란 @cdism
3,2022-02-25 13:07:04,750628453866274870,NaN,saypark3824,https://blog.naver.com/saypark3824,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjVfNTkg/MD...,정답 : 3번 입니다 @최준원 학생기자
4,2022-02-25 13:07:35,750628486162415983,NaN,리체,https://blog.naver.com/litvice,0,0,0,NaN,3번~~~^^


In [1100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       1382 non-null   object
 1   고유번호      1382 non-null   object
 2   답글구분      94 non-null     object
 3   닉네임       1382 non-null   object
 4   작성자URL    1382 non-null   object
 5   공감수       1382 non-null   int64 
 6   링크수       1382 non-null   int64 
 7   답글수       1382 non-null   int64 
 8   첨부이미지URL  233 non-null    object
 9   내용        1382 non-null   object
dtypes: int64(3), object(7)
memory usage: 108.1+ KB


### df 확인
* 고유번호 : undefined 제거
* 닉네임 : 비밀댓글 제거
* 답글 구분 : 비밀댓글 nan값으로 교체

#### 고유번호에서 undefined 제거

In [1101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       1382 non-null   object
 1   고유번호      1382 non-null   object
 2   답글구분      94 non-null     object
 3   닉네임       1382 non-null   object
 4   작성자URL    1382 non-null   object
 5   공감수       1382 non-null   int64 
 6   링크수       1382 non-null   int64 
 7   답글수       1382 non-null   int64 
 8   첨부이미지URL  233 non-null    object
 9   내용        1382 non-null   object
dtypes: int64(3), object(7)
memory usage: 108.1+ KB


In [1102]:
df[df['고유번호'] == 'undefined']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
550,2022-02-27 17:29:16,undefined,NaN,란나,https://blog.naver.com/kiboda,0,0,3,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
601,2022-02-27 17:29:16,undefined,NaN,란나,https://blog.naver.com/kiboda,0,0,3,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,


In [1103]:
undefined_index = df[df['고유번호'] == 'undefined'].index
undefined_index

Int64Index([550, 601], dtype='int64')

In [1104]:
df.iloc[undefined_index[0]-1:undefined_index[0]+2, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
549,2022-02-27 17:29:02,750830872990580750,NaN,란나,https://blog.naver.com/kiboda,0,1,0,NaN,정답: 3번인천광역시의회 항상 응원합니다♡ @사랑님 https://m.blog....
550,2022-02-27 17:29:16,undefined,NaN,란나,https://blog.naver.com/kiboda,0,0,3,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
551,2022-02-27 17:29:16,750830888173961351,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,


In [1105]:
df.iloc[undefined_index[1]-1:undefined_index[1]+2, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
600,2022-02-28 16:25:50,750919573829583329,NaN,쬬리,https://blog.naver.com/honey_ong,0,1,0,NaN,3번 기존이웃입니다 https://m.blog.naver.com/honey_ong...
601,2022-02-27 17:29:16,undefined,NaN,란나,https://blog.naver.com/kiboda,0,0,3,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
602,2022-02-27 17:29:16,750830888173961351,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,


In [1106]:
df2 = df.drop(index=undefined_index).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380 entries, 0 to 1379
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       1380 non-null   object
 1   고유번호      1380 non-null   object
 2   답글구분      94 non-null     object
 3   닉네임       1380 non-null   object
 4   작성자URL    1380 non-null   object
 5   공감수       1380 non-null   int64 
 6   링크수       1380 non-null   int64 
 7   답글수       1380 non-null   int64 
 8   첨부이미지URL  231 non-null    object
 9   내용        1380 non-null   object
dtypes: int64(3), object(7)
memory usage: 107.9+ KB


In [1107]:
df2.iloc[undefined_index[0]-1:undefined_index[0]+2, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
549,2022-02-27 17:29:02,750830872990580750,NaN,란나,https://blog.naver.com/kiboda,0,1,0,NaN,정답: 3번인천광역시의회 항상 응원합니다♡ @사랑님 https://m.blog....
550,2022-02-27 17:29:16,750830888173961351,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
551,2022-02-27 17:29:28,750830900253556981,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTQ3/MD...,


In [1108]:
df2.iloc[undefined_index[1]-1:undefined_index[1]+2, :]

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
600,2022-02-27 17:29:16,750830888173961351,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTY1/MD...,
601,2022-02-27 17:29:28,750830900253556981,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfMTQ3/MD...,
602,2022-02-27 17:29:38,750830912618365258,답글,란나,https://blog.naver.com/kiboda,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjdfNDUg/MD...,


#### 닉네임에서 비밀댓글 제거

In [1109]:
df2[df2['닉네임'] == '비밀댓글']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
91,2022-02-25 15:34:45,750637968544236341,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
141,2022-02-25 15:34:45,750637968544236341,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
174,2022-02-25 17:56:26,750647096389927242,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
181,2022-02-25 19:45:54,750654147266609902,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
190,2022-02-25 20:32:59,750657180503376843,비밀댓글,비밀댓글,비밀댓글,0,0,1,NaN,비밀댓글
191,2022-02-25 20:33:03,750657185435877425,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
224,2022-02-25 17:56:26,750647096389927242,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
231,2022-02-25 19:45:54,750654147266609902,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
240,2022-02-25 20:32:59,750657180503376843,비밀댓글,비밀댓글,비밀댓글,0,0,1,NaN,비밀댓글
241,2022-02-25 20:33:03,750657185435877425,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글


In [1110]:
secret_reply_index = df2[df2['닉네임'] == '비밀댓글'].index
secret_reply_index

Int64Index([  91,  141,  174,  181,  190,  191,  224,  231,  240,  241,  256,
             306,  357,  407,  559,  609,  772,  822, 1185, 1186, 1187, 1188,
            1189, 1196, 1235, 1236, 1237, 1238, 1239, 1246, 1292, 1296, 1342,
            1346, 1379],
           dtype='int64')

In [1111]:
df3 = df2.drop(index=secret_reply_index).reset_index(drop=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345 entries, 0 to 1344
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       1345 non-null   object
 1   고유번호      1345 non-null   object
 2   답글구분      59 non-null     object
 3   닉네임       1345 non-null   object
 4   작성자URL    1345 non-null   object
 5   공감수       1345 non-null   int64 
 6   링크수       1345 non-null   int64 
 7   답글수       1345 non-null   int64 
 8   첨부이미지URL  231 non-null    object
 9   내용        1345 non-null   object
dtypes: int64(3), object(7)
memory usage: 105.2+ KB


In [1112]:
df3['답글구분'].unique()

array([nan, '답글', '비밀댓글'], dtype=object)

#### 답글구분에서 비밀댓글 nan 값으로 교체

In [1113]:
df3[df3['답글구분'] == '비밀댓글']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
1166,2022-03-04 21:24:40,751309911094198722,비밀댓글,단단해진나,https://blog.naver.com/snowdropsun,0,4,5,NaN,정답 : 3번 https://m.blog.naver.com/snowdropsun/2...
1210,2022-03-04 21:24:40,751309911094198722,비밀댓글,단단해진나,https://blog.naver.com/snowdropsun,0,4,5,NaN,정답 : 3번 https://m.blog.naver.com/snowdropsun/2...
1264,2022-03-04 23:12:05,751316830286512183,비밀댓글,금덩이,https://blog.naver.com/pinkangel9999,0,4,1,NaN,3번인천광역시의회 멋진 정책 법안을 많이 만들어 주셔서 시간가는 줄 모르고 재미나게...
1312,2022-03-04 23:12:05,751316830286512183,비밀댓글,금덩이,https://blog.naver.com/pinkangel9999,0,4,1,NaN,3번인천광역시의회 멋진 정책 법안을 많이 만들어 주셔서 시간가는 줄 모르고 재미나게...


In [1114]:
index3 = df3[df3['답글구분'] == '비밀댓글'].index
index3

Int64Index([1166, 1210, 1264, 1312], dtype='int64')

In [1115]:
df4 = df3.replace( {'답글구분':'비밀댓글'}, nan )

In [1116]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345 entries, 0 to 1344
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       1345 non-null   object
 1   고유번호      1345 non-null   object
 2   답글구분      55 non-null     object
 3   닉네임       1345 non-null   object
 4   작성자URL    1345 non-null   object
 5   공감수       1345 non-null   int64 
 6   링크수       1345 non-null   int64 
 7   답글수       1345 non-null   int64 
 8   첨부이미지URL  231 non-null    object
 9   내용        1345 non-null   object
dtypes: int64(3), object(7)
memory usage: 105.2+ KB


In [1117]:
final_df = df4.copy()

## 파일로 출력

In [1118]:
crawled_folder_path

'c:/data/Wantreez/Crawling/incheon/crawled_data/naver_blog'

In [1119]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
today_date

'20220421_161339'

In [1120]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
file_name = f'naver_blog_{today_date}.xlsx'
final_df.to_excel(crawled_folder_path + '/' + file_name, index=False, encoding='utf-8')
print(f'{file_name} 출력완료')

naver_blog_20220421_161340.xlsx 출력완료


# 코드 마지막